In [9]:
import json

In [51]:
class Load_Preprocess():
    def __init__(self):
        self.Label2num = {'Entailment': 1, "Contradiction": 0}
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
        self.train_data = self.load_Processed_Data(self.load_Processed_cTAKES('ctakes/Train_Statements_cTAKES_Processed.json'), 'train')
        self.dev_data = self.load_Processed_Data(self.load_Processed_cTAKES('ctakes/Dev_Statements_cTAKES_Processed.json'), 'dev')
        self.test_data = self.load_Processed_Data(self.load_Processed_cTAKES('ctakes/Test_Statements_cTAKES_Processed.json'), 'test')
        
        self.statements_embeddings = 0
        self.section_embeddings = 0
        self.trail1_embeddings = 0
        self.trail2_embeddings = 0
        self.preferred_text_embeddings = 0

    def load_Processed_cTAKES(self, split):
        ctakes_tokens_path = split
        with open(ctakes_tokens_path) as json_file:
            ctakes_tokens = json.load(json_file)

        preferred_text_dict = dict()
        for i in range(len(ctakes_tokens)):
            cm_dict = ctakes_tokens[i]['clinical_mention']
            temp_list = []
            for key in cm_dict.keys():
                try:
                    temp_list.append(cm_dict[key][0]['preferredText'])
                except:
                    continue
            preferred_text_dict[ctakes_tokens[i]['UUID'][0]] = temp_list
        return preferred_text_dict

    def load_Processed_Data(self, ctakes, split):
        preferred_text = []
        statement = []
        trail1 = []
        trail2 = []
        section = []
        label = []
        
        with open(f"training_data/{split}" + ".json") as file:
            data = json.load(file)
            uuid_list = list(data.keys())

        for id in uuid_list:
            statement.append(data[id]['Statement'])
            if split != 'test':
                label.append(self.Label2num[data[id]['Label']])
            section.append(data[id]['Section_id'])
        
            with open(f"training_data/CT json/{data[id]['Primary_id']}" + ".json") as file: 
                ct = json.load(file)
                trail1.append(self.join_list(ct[data[id]['Section_id']]))
                
            if data[id]['Type'] == "Comparison":  
                with open(f"training_data/CT json/{data[id]['Secondary_id']}" + ".json") as file:
                    ct = json.load(file)
                    trail2.append(self.join_list(ct[data[id]['Section_id']]))
            else:
                trail2.append("_")
                
            preferred_text.append(','.join(ctakes[id]))
            
        if split!= 'test':
            return {'preferred_text':preferred_text, 'statement':statement, 'trail1':trail1, 'trail2':trail2, 'section':section, 'label':label}
        else:
            return {'preferred_text':preferred_text, 'statement':statement, 'trail1':trail1, 'trail2':trail2, 'section':section}
    
    def join_list(self, sentences):
        return ", ".join([sent.strip() for sent in sentences])

In [240]:
train_data = Load_Preprocess().train_data
test_data = Load_Preprocess().test_data
dev_data = Load_Preprocess().dev_data

In [241]:
train_label_list = train_data['label']
dev_label_list = dev_data['label']

In [242]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import torch
from transformers import LongformerModel, AutoTokenizer

# Load pre-trained model tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')

model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

C:\Users\Fuzel Shaik\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Fuzel Shaik\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [245]:
df = pd.DataFrame(train_data)
df = df[['preferred_text','statement','trail1','trail2','section']]
df

,preferred_text,statement,trail1,trail2,section
0,"Oral cavity,Primary operation,Oral Dosage Form",All the primary trial participants do not rece...,"INTERVENTION 1:, Diagnostic (FLT PET), Patient...","INTERVENTION 1:, Arm A, Patients receive oral ...",Intervention
1,"Blood Platelets,Platelet Count measurement,Hem...","Patients with Platelet count over 100,000/mm¬¨...","DISEASE CHARACTERISTICS:, Histologically or cy...",_,Eligibility
2,"Related personal status,Heart,Primary operation",Heart-related adverse events were recorded in ...,"Adverse Events 1:, Total: 5/32 (15.63%), Febri...","Adverse Events 1:, Total: 285/752 (37.90%), An...",Adverse Events
3,"Both breasts,Breast Carcinoma,Primary operation",Adult Patients with histologic confirmation of...,"Inclusion Criteria:, Patients with histologic ...",_,Eligibility
4,Laser Therapy,Laser Therapy is in each cohort of the primary...,"INTERVENTION 1:, Laser Therapy Alone, therapis...","INTERVENTION 1:, Part A Abemaciclib: HR+, HER2...",Intervention
...,...,...,...,...,...
1695,"Liver function,Blood,Disease,Primary operation","Adequate blood, kidney, and hepatic function a...","Inclusion Criteria:, Postmenopausal women, Sta...","Inclusion Criteria:, Pathologically confirmed ...",Eligibility
1696,"Primary operation,dalotuzumab",The Ridaforolimus + Dalotuzumab + Exemestane g...,"Outcome Measurement:, 1. Progression-free Surv...",_,Results
1697,"Patient position finding,Primary operation",The only difference between the interventions ...,"INTERVENTION 1:, Prone, Prone position, INTERV...",_,Intervention
1698,"Leukocytes,White Blood Cell Count procedure",Patients must have a white blood cell count ab...,"DISEASE CHARACTERISTICS:, Histologically confi...",_,Eligibility


In [248]:
def get_bert_embeddings(text):
    #input_ids = tokenizer.encode(text, padding=True, truncation=True, max_length=1024, add_special_tokens=True, return_tensors='pt')
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    global_attention_mask = torch.zeros(input_ids.shape, dtype=torch.long, device=input_ids.device)
    token_len = len(text.split())
    N = int(token_len*0.1)
    attention_list = np.linspace(0, token_len, N, endpoint=True)
    global_attention_mask[
        :,
        attention_list,
    ] = 1

    with torch.no_grad():
        outputs = model(input_ids,attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    return outputs.last_hidden_state[:,0,:].numpy()


def embedder(data):

    df = pd.DataFrame(data)
    df = df[['preferred_text','statement','trail1','trail2','section']]

    embedding_matrix = np.zeros((df.shape[0], 5, 768))

    for i in range(df.shape[0]):
        preferred_text_embeddings = get_bert_embeddings(df['preferred_text'].iloc[i])
        statement_embeddings = get_bert_embeddings(df['statement'].iloc[i])
        trail1_embeddings = get_bert_embeddings(df['trail1'].iloc[i])
        trail2_embeddings = get_bert_embeddings(df['trail2'].iloc[i])
        section_embeddings = get_bert_embeddings(df['section'].iloc[i])
        stacked_matrix = np.vstack([preferred_text_embeddings, statement_embeddings, trail1_embeddings, trail2_embeddings, section_embeddings])
        #stacked_matrix_transpose = stacked_matrix.T
        embedding_matrix[i, :, :] = stacked_matrix

    return embedding_matrix

In [249]:
dev_embedded_vector = embedder(dev_data)
dev_embedded_vector

array([[[-5.22220209e-02,  9.73302871e-02, -1.63271725e-02, ...,
         -1.09394550e-01, -2.58570928e-02, -3.53406146e-02],
        [-4.64698449e-02,  6.45281151e-02, -4.72743511e-02, ...,
         -9.51061398e-02, -4.10685912e-02, -5.67037761e-02],
        [-1.99822709e-03,  5.82316816e-02, -7.34675825e-02, ...,
          4.04712558e-03,  1.29969865e-02, -2.08499357e-02],
        [-6.27781227e-02,  4.58259694e-02,  9.64748580e-03, ...,
         -1.25837997e-01, -1.27292825e-02, -4.85871360e-02],
        [-2.58054882e-02,  1.11741468e-01, -1.35565624e-02, ...,
         -1.01113915e-01, -5.30097038e-02, -2.67949253e-02]],

       [[-5.80525920e-02,  9.74157453e-02, -2.99124978e-02, ...,
         -5.45478463e-02, -3.58684510e-02,  6.55969232e-03],
        [-8.71125758e-02,  8.19439292e-02, -1.71879828e-02, ...,
         -6.28861040e-03, -4.77507226e-02, -7.25038350e-02],
        [-9.09730792e-04,  6.56911358e-02, -3.38833630e-02, ...,
          7.46725500e-03, -3.94235253e-02, -4.31492

In [250]:
train_embedded_vector = embedder(train_data)
train_embedded_vector.shape

(1700, 5, 768)

In [256]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_lstm_layers, num_heads, output_size):
        super(LSTMModel, self).__init__()

        self.lstm_layers = nn.ModuleList([nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, bidirectional=True) for _ in range(num_lstm_layers)])

        self.self_attention = nn.MultiheadAttention(embed_dim=2*hidden_size, num_heads=num_heads)
        
        self.fc = nn.Linear(2*hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out = x
        for lstm_layer in self.lstm_layers:
            lstm_out, _ = lstm_layer(lstm_out)

        lstm_out = self.relu(lstm_out) 

        attn_output, _ = self.self_attention(lstm_out.transpose(0, 1), lstm_out.transpose(0, 1), lstm_out.transpose(0, 1))
        attn_output = attn_output.transpose(0, 1) 

        last_hidden_state = attn_output.mean(dim=1)  
        output = self.fc(last_hidden_state)
        return output

batch_size = 64
seq_len = 5
input_size = 768
output_size = 1

train_tensor_data = TensorDataset(torch.from_numpy(train_embedded_vector).float(), torch.tensor(np.array(train_label_list)).float())
train_loader = DataLoader(train_tensor_data, shuffle=True, batch_size=batch_size, drop_last=True)

hidden_size = 128
num_lstm_layers = 2
num_heads = 4

lstm_model = LSTMModel(input_size, hidden_size, num_lstm_layers, num_heads, output_size)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    lstm_model.train()
    total_loss = 0.0
    predictions = []
    true_labels = []

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = lstm_model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy() )
        true_labels.extend(labels.cpu().numpy())

    mean_loss = total_loss / len(train_loader)
    predictions = [1 if p > 0.5 else 0 for p in predictions]
    
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {mean_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch 1/20, Train Loss: 0.6949, Accuracy: 0.4826, Precision: 0.4766, Recall: 0.2915, F1 Score: 0.3617
Epoch 2/20, Train Loss: 0.6933, Accuracy: 0.4760, Precision: 0.4672, Recall: 0.2888, F1 Score: 0.3569
Epoch 3/20, Train Loss: 0.6933, Accuracy: 0.4994, Precision: 0.4994, Recall: 1.0000, F1 Score: 0.6661
Epoch 4/20, Train Loss: 0.6934, Accuracy: 0.4808, Precision: 0.4845, Recall: 0.6422, F1 Score: 0.5523
Epoch 5/20, Train Loss: 0.6933, Accuracy: 0.4994, Precision: 0.4688, Recall: 0.0362, F1 Score: 0.0672
Epoch 6/20, Train Loss: 0.6936, Accuracy: 0.4730, Precision: 0.4688, Recall: 0.3962, F1 Score: 0.4294


C:\Users\Fuzel Shaik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 7/20, Train Loss: 0.6934, Accuracy: 0.5036, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


C:\Users\Fuzel Shaik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 8/20, Train Loss: 0.6932, Accuracy: 0.4988, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 9/20, Train Loss: 0.6932, Accuracy: 0.4796, Precision: 0.4531, Recall: 0.2096, F1 Score: 0.2867
Epoch 10/20, Train Loss: 0.6932, Accuracy: 0.4868, Precision: 0.4805, Recall: 0.3554, F1 Score: 0.4086
Epoch 11/20, Train Loss: 0.6932, Accuracy: 0.4892, Precision: 0.4917, Recall: 0.5659, F1 Score: 0.5262
Epoch 12/20, Train Loss: 0.6932, Accuracy: 0.5006, Precision: 0.5156, Recall: 0.0396, F1 Score: 0.0736
Epoch 13/20, Train Loss: 0.6933, Accuracy: 0.4892, Precision: 0.4940, Recall: 0.7962, F1 Score: 0.6097
Epoch 14/20, Train Loss: 0.6932, Accuracy: 0.5018, Precision: 0.5089, Recall: 0.2724, F1 Score: 0.3549
Epoch 15/20, Train Loss: 0.6932, Accuracy: 0.4898, Precision: 0.4943, Recall: 0.8355, F1 Score: 0.6212
Epoch 16/20, Train Loss: 0.6933, Accuracy: 0.4916, Precision: 0.4813, Recall: 0.1847, F1 Score: 0.2669


C:\Users\Fuzel Shaik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 17/20, Train Loss: 0.6932, Accuracy: 0.5000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch 18/20, Train Loss: 0.6932, Accuracy: 0.4868, Precision: 0.4905, Recall: 0.6791, F1 Score: 0.5696
Epoch 19/20, Train Loss: 0.6933, Accuracy: 0.4922, Precision: 0.4781, Recall: 0.1841, F1 Score: 0.2659
Epoch 20/20, Train Loss: 0.6932, Accuracy: 0.4784, Precision: 0.4688, Recall: 0.3245, F1 Score: 0.3835


In [271]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            
            labels = labels.squeeze()
            
            loss = criterion(outputs.squeeze(), labels.float())
            total_loss += loss.item()

            predictions.append(torch.sigmoid(outputs).cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    mean_loss = total_loss / len(dataloader)
    predictions = [1 if p > 0.5 else 0 for p in predictions]

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    return mean_loss, accuracy, precision, recall, f1


criterion = nn.BCEWithLogitsLoss()

dev_tensor_data = TensorDataset(torch.from_numpy(dev_embedded_vector).float(), torch.tensor(np.array(dev_label_list)).float())
dev_loader = DataLoader(dev_tensor_data, shuffle=True, drop_last=True)

val_loss, val_accuracy, val_precision, val_recall, val_f1 = evaluate(lstm_model, dev_loader, criterion)
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}")

Validation Loss: 0.6931, Accuracy: 0.5000, Precision: 0.5000, Recall: 1.0000, F1 Score: 0.6667


In [ ]:
test_embedded_vector = embedder(test_data)
test_embedded_vector

In [238]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class BERTLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BERTLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden_state = lstm_out[:, -1, :]
        output = self.fc(last_hidden_state)
        return output


batch_size = 512
train_tensor_data = TensorDataset(torch.from_numpy(train_embedded_vector).float(), torch.tensor(train_label_list).float())
train_loader = DataLoader(train_tensor_data, shuffle=True, batch_size=batch_size, drop_last=True)


input_size = 768  
hidden_size = 128
num_layers = 2
output_size = 1


lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)


num_epochs = 10

for epoch in range(num_epochs):
    lstm_model.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = lstm_model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    mean_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {mean_loss:.4f}")

Epoch 1/10, Train Loss: 0.6933
Epoch 2/10, Train Loss: 0.6933
Epoch 3/10, Train Loss: 0.6930
Epoch 4/10, Train Loss: 0.6933
Epoch 5/10, Train Loss: 0.6932
Epoch 6/10, Train Loss: 0.6931
Epoch 7/10, Train Loss: 0.6928
Epoch 8/10, Train Loss: 0.6931
Epoch 9/10, Train Loss: 0.6929
Epoch 10/10, Train Loss: 0.6927


In [232]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            total_loss += loss.item()

            predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    mean_loss = total_loss / len(dataloader)
    predictions = [1 if p > 0.5 else 0 for p in predictions]

    accuracy = accuracy_score(true_labels, predictions)

    return mean_loss, accuracy

In [239]:
batch_size = 64
dev_tensor_data = TensorDataset(torch.from_numpy(dev_embedded_vector).float(), torch.tensor(dev_label_list).float())
dev_loader = DataLoader(dev_tensor_data, shuffle=True, batch_size=batch_size, drop_last=True)

val_loss, val_accuracy = evaluate(lstm_model, dev_loader, criterion)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Validation Loss: 0.6931, Validation Accuracy: 0.5052


In [243]:
y_test = torch.randint(0, 2, (300,))

In [244]:
y_test

tensor([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,

In [141]:
temp = train_embedded_vector.reshape((1700, -1))
temp.shape

(1700, 3840)

In [139]:
temp[1,0]

-0.10078100115060806

In [172]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

In [170]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [176]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [181]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="LSTM"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 1
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.time()
    for i in test_x.shape[0]:
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [164]:
batch_size = 512
train_data = TensorDataset(torch.from_numpy(train_embedded_vector), torch.from_numpy(np.array(train_label_list)))

In [165]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [166]:
train_loader.dataset.tensors

(tensor([[[-0.3775, -0.4976, -0.9053, -0.8667, -0.4670],
          [ 0.2264, -0.2347, -0.3397, -0.4702,  0.0931],
          [-0.1990,  0.0105,  0.3920,  0.0222, -0.1040],
          ...,
          [-0.2423, -0.4277, -0.1000, -0.1339,  0.0098],
          [ 0.0702,  0.3440,  0.0351,  0.1352,  0.0057],
          [ 0.4793,  0.3744,  0.5974,  0.5052,  0.2998]],
 
         [[-0.1008, -0.0921, -0.5842, -0.2127, -0.2874],
          [-0.1103, -0.4184,  0.1822,  0.2817,  0.1403],
          [-0.3339, -0.3547,  0.2090,  0.0348, -0.0888],
          ...,
          [-0.6115, -0.1707, -0.2617, -0.3424, -0.1138],
          [-0.3047,  0.4071, -0.0997,  0.1727, -0.0028],
          [ 0.1047,  0.5937,  0.2047,  0.2925,  0.3569]],
 
         [[-0.1764, -0.3942, -0.1573, -0.3861, -0.5852],
          [-0.0287, -0.5400, -0.0861,  0.0642,  0.2510],
          [-0.2478, -0.8151, -0.3830, -0.3143, -0.4083],
          ...,
          [-0.1417, -0.1896, -0.1548, -0.1320, -0.3164],
          [ 0.0489,  0.6504, -0.1846,

In [168]:
next(iter(train_loader))[0].shape[2]

5

In [182]:
lr = 0.001
#gru_model = train(train_loader, lr, model_type="GRU")
Lstm_model = train(train_loader, lr, model_type="LSTM")

Starting Training of LSTM model
Epoch 1/5 Done, Total Loss: 0.4361451268196106
Total Time Elapsed: 29.382312059402466 seconds
Epoch 2/5 Done, Total Loss: 0.36259178320566815
Total Time Elapsed: 29.700926303863525 seconds
Epoch 3/5 Done, Total Loss: 0.2799377739429474
Total Time Elapsed: 30.123373985290527 seconds
Epoch 4/5 Done, Total Loss: 0.27400384346644086
Total Time Elapsed: 30.800617933273315 seconds
Epoch 5/5 Done, Total Loss: 0.2705318530400594
Total Time Elapsed: 29.678523302078247 seconds
Total Training Time: 149.68575358390808 seconds


In [188]:
def evaluate_lstm_model(model, X_test, y_test):
    model.eval()
    criterion = nn.MSELoss()

    with torch.no_grad():
        total_loss = 0.0
        predictions = []

        for inputs, labels in zip(X_test, y_test):
            inputs, labels = torch.tensor(inputs, dtype=torch.float32), torch.tensor(np.array(labels), dtype=torch.float32)
            
            h = model.init_hidden(64)
            
            h = tuple([e.data for e in h])

            # Forward pass
            outputs = model(inputs,h)

            # Compute the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions.append(outputs.numpy())

        mean_loss = total_loss / len(X_test)
        print(f"Mean Loss on Test Data: {mean_loss:.4f}")

        return np.concatenate(predictions, axis=0)

In [189]:
predictions = evaluate_lstm_model(Lstm_model, dev_embedded_vector, dev_label_list)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [190]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Input shape: (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        
        # Extract the last time step's output
        last_hidden_state = lstm_out[:, -1, :]
        
        # Fully connected layer
        output = self.fc(last_hidden_state)
        
        return output

In [191]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming you have a training dataset X_train of shape (num_samples, seq_len, input_size)
# and corresponding labels y_train of shape (num_samples, output_size)

def train_lstm_model(model, X_train, y_train, num_epochs=10, learning_rate=0.001):
    criterion = nn.MSELoss()  # Mean Squared Error loss for regression task
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in zip(X_train, y_train):
            inputs, labels = torch.tensor(inputs, dtype=torch.float32), torch.tensor(np.array(labels), dtype=torch.float32)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(X_train)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    print("Training finished.")

In [194]:
input_size = 5 # Size of each input feature
hidden_size = 128  # Number of hidden units in the LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Size of the output

lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

train_lstm_model(lstm_model, train_embedded_vector, train_label_list, num_epochs=10, learning_rate=0.001)

IndexError: too many indices for tensor of dimension 2